In [ ]:
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import numpy as np
import parambokeh

from earthsim.annotators import (
    PolyAnnotator, PointAnnotator, PolyAndPointAnnotator,
    GeoAnnotator, PointWidgetAnnotator
)

hv.extension('bokeh')

This notebook documents the usage and design of high-level ``GeoAnnotator`` classes, which make it easy to draw, edit and annotate polygon, multi-line and point data on top of a map. While the existing Helper classes already provide complex functionality they should be considered templates, demonstrating useful patterns which can be adapted to more specific requirements and extended. The ``GeoAnnotator`` classes build on bokeh [Drawing Tools](Drawing_Tools.ipynb), connected to HoloViews streams to access the drawn data in Python.

# GeoAnnotator

The ``GeoAnnotator`` allows drawing polygons and points on top of a tile source and syncing the drawn data back to Python. It does this by attaching ``PointDraw``, ``PolyDraw`` and ``VertexEdit`` streams to the points and polygons, which in turn add the corresponding tools.

In [ ]:
helper = GeoAnnotator()
helper.pprint()
helper.view()

## Accessing the stream data

The data drawn in the above plot is automatically synced to Python, we can easily access it on the two stream classes:

#### Polygon

The data can be accessed directly on the stream and matches the format accepted by a bokeh ``ColumnDataSource`` and will be in Web Mercator coordinates:

In [ ]:
helper.poly_stream.data

For ease of use the data can also be accessed using the ``element`` attribute:

In [ ]:
helper.poly_stream.element

This also makes it easy to project the data:

In [ ]:
gv.operation.project_path(helper.poly_stream.element, projection=ccrs.PlateCarree()).dframe()

Finally we can also get a dynamic plot of the Polygon data, which updates whenever the stream changes:

In [ ]:
helper.poly_stream.dynamic

#### Points

The same functionality is also available for the ``point_stream``:

In [ ]:
helper.point_stream.element

## PointAnnotator

The ``PointAnnotator`` is an extension of the ``GeoAnnotator`` which also adds support for annotating the points with the help of a table. Whenever a point is added by tapping on the plot an entry will appear in the table below the plot allowing you to edit the specified ``point_columns``. By default we can edit the 'Size' column.

After selecting the Point Draw Tool you can tap anywhere to draw points, drag the points around and delete them with backspace. Whenever a point is added it will appear in the table, by tapping on the empty 'Size' cells you can enter a value, which will also be synced back to Python. Selecting one or more rows in the table will highlight the corresponding points.

In [ ]:
%%opts Points (size=10) [tools=['hover']]
helper = PointAnnotator(point_columns=['Size'])
helper.pprint()
helper.view()

#### Accessing the data in Python

Once again we can access the annotated points in Python by looking at the ``point_table_stream``:

In [ ]:
gv.operation.project_points(helper.point_stream.element, projection=ccrs.PlateCarree()).dframe()

## PolyAnnotator

The ``PolyAnnotator`` works much the same as the ``PointAnnotator`` except that it allows us to annotate polygons. As before whenever a polygon is added, this time using the Polygon Draw tool it will appear in the table below, selecting a row will highlight the corresponding polygon.

In [ ]:
helper = PolyAnnotator()
helper.pprint()
helper.view()

## PolyAndPointAnnotator

The ``PolyAndPointAnnotator`` combines the ``PointAnnotator`` and ``PolyAnnotator`` showing two tables to add annotations to both the points and polygons.

In [ ]:
%%opts Polygons (color='red' alpha=0.5 selection_alpha=0.8 nonselection_alpha=0.2) 
%%opts Points (size=10 nonselection_alpha=0.5) Layout [shared_datasource=True]
helper = PolyAndPointAnnotator()
helper.pprint()
helper.view()

In [ ]:
helper.poly_table_stream

## WidgetAnnotator

The ``WidgetAnnotator`` takes a different approach to annotating points. Instead of annotating points by editing the Table directly it allows adding points to a number of predefined groups.

1. Add some points
2. Select a subset of the points by tapping on them or using the box_select tool
3. Select a group to assign to the points from the dropdown menu
4. Click the add button

The indexes of the points assigned to each group can be seen in the table.

In [ ]:
%%opts Points [tools=['box_select']] (size=10 line_color='black')
annotator = PointWidgetAnnotator(['A', 'B', 'C'])
annotator.pprint()
parambokeh.Widgets(annotator)
annotator.view()

We can also view the annotated points separately:

In [ ]:
%%opts Layout [shared_datasource=True] Points (size=10)
points = annotator.annotated_points()
points + points.table()